In [ ]:
# 전기차 충전소 자료 정리
# 전기차 충전소 자료 불러오기
charge1 = pd.read_csv("../data/Intro/환경부_전기자동차 급속충전기 보급 현황_20211231.csv")
charge2 = pd.read_csv("../data/Intro/환경부_전기자동차 완속충전기 보급 현황_20211231.csv")

# 칼럼명 변경
charge1.rename(columns={"급속충전기 보급 수량":"급속 수량"}, inplace=True)
charge2.rename(columns={"완속충전기 보급수량(단위-기)":"완속 수량"}, inplace=True)

# 충전소 완속+급속 총수량 데이터 프레임 만들기
charge1["완속 수량"]=charge2["완속 수량"]
charge1=charge1[["년도", "급속 수량", "완속 수량"]]
charge1["총 수량"]=charge1["급속 수량"]+charge1["완속 수량"]






# 연료별 자동차 등록대수 자료 정리
# 전기차 등록대수 데이터 불러오기
car_info = pd.read_csv("../data/자동차+등록현황(연료별)_20221025141634.csv")
# "계"행 삭제
car_info=car_info.drop(columns=["차종별(1)","연료별(1)"],axis=1)
# 컬럼명 변경
car_info.rename(columns={"차종별(2)":"차종",
                        "연료별(2)":"연료"}, inplace=True)
# "소계"열 삭제
car_info=car_info[car_info["차종"]!="소계"]

# "소계"행 삭제
car_info = car_info[car_info["연료"]!="소계"]

# 칼럼중 -를 0으로 대체
car_info = car_info.replace("-","0")

# 칼럼중 공백 0으로 대체
car_info =car_info.fillna(0)

# 사용할 컬럼만 남기고 삭제 
car_info=car_info[["차종","연료","2017. 01","2017. 12",
                   "2018. 01","2018. 12",
                   "2019. 01","2019. 12",
                   "2020. 01","2020. 12",
                   "2021. 01","2021. 12"]]

# 월별 차량 대수 숫자형 변환
car_info["2017. 01"]=pd.to_numeric(car_info["2017. 01"])
car_info["2017. 12"]=pd.to_numeric(car_info["2017. 12"])
car_info["2018. 01"]=pd.to_numeric(car_info["2018. 01"])
car_info["2018. 12"]=pd.to_numeric(car_info["2018. 12"])
car_info["2019. 01"]=pd.to_numeric(car_info["2019. 01"])
car_info["2019. 12"]=pd.to_numeric(car_info["2019. 12"])
car_info["2020. 01"]=pd.to_numeric(car_info["2020. 01"])
car_info["2020. 12"]=pd.to_numeric(car_info["2020. 12"])
car_info["2021. 01"]=pd.to_numeric(car_info["2021. 01"])
car_info["2021. 12"]=pd.to_numeric(car_info["2021. 12"])

# 연도별 차량등록대수 변화량 행추가
car_info["2017"]=car_info["2017. 12"]-car_info["2017. 01"]
car_info["2018"]=car_info["2018. 12"]-car_info["2018. 01"]
car_info["2019"]=car_info["2019. 12"]-car_info["2019. 01"]
car_info["2020"]=car_info["2020. 12"]-car_info["2020. 01"]
car_info["2021"]=car_info["2021. 12"]-car_info["2021. 01"]

# 차종, 연료, 연도 별 증감량 만들기
car_YoY = car_info[["차종","연료","2017","2018","2019","2020","2021"]].set_index(["차종","연료"])
# 연료별 연도별 연말 등록대수 확인
car_reg = car_info[["차종","연료","2017. 12","2018. 12","2019. 12","2020. 12","2021. 12"]].set_index(["차종","연료"])





# 연도별 자동차 누적 등록대수 자료 정리
# 전기차 등록대수만 추출
car_info1= car_info.loc[car_info["연료"]=="전기",]
# 연도별 등록 수량만 추출
car_info1=car_info1[["차종","2017. 12", "2018. 12", "2019. 12","2020. 12","2021. 12"]]
# 충전소랑 형태 맞추기
car_info1=car_info1.transpose()
car_info1.rename(columns=car_info1.iloc[0],inplace=True)
car_info1 = car_info1.drop(car_info1.index[0])
car_info1["전기차 등록대수"]=car_info1["승용차"]+car_info1["승합차"]+car_info1["화물차"]+car_info1["특수차"]

# 년도 칼럼명 변경
car_info1["년도"]=car_info1.index
car_info2=car_info1[["년도","전기차 등록대수"]]
car_info2.index=[1,2,3,4,5]
car_info2

# 전기차 충전소 자료에 전기차 등록대수 자료 합치기
charge1["전기차 등록대수"]=car_info2["전기차 등록대수"]
charge1





# 그래프 그리기
fig = plt.figure(figsize=(8,8)) ## Figure 생성 
fig.set_facecolor('white') ## Figure 배경색 지정
ax1 = fig.add_subplot() ## axes 생성
 
colors = sns.color_palette('summer', len(charge1["년도"])) ## 바 차트 색상
 
xtick_label_position = list(charge1["년도"]) ## x축 눈금 라벨이 표시될 x좌표
ax1.set_xticks(xtick_label_position) ## x축 눈금 
ax1.set_xticklabels(charge1["년도"]) ## x축 눈금 라벨
ax1.set_ylabel("충전소")
ax1.bar(xtick_label_position, charge1["총 수량"], color=colors) ## 바차트 출력
 
color = 'blue'
ax2 = ax1.twinx() ## 새로운 axis 생성
ax2.plot(xtick_label_position, charge1["전기차 등록대수"], color=color, linestyle='--', marker='o') ## 선 그래프 
ax2.set_ylabel("등록 대수")
ax2.tick_params(axis='y', labelcolor=color) ## 눈금 라벨 색상 지정
 
plt.title('전기차 충전소 및 등록대수', fontsize=20)
plt.show()